# Client Presentation 2: Regression Models

This notebook will take a dataset, remove multicollinearity and near-zero variance, and create the KNN Regression Model.

## 1. Read in Data, Import Packages

In [2]:
import numpy as np
import pandas as pd

pts_df = pd.read_csv('/Users/dB/Documents/repos/github/bint-capstone/data-sources/df_scaled_assists.csv')
to_df = pd.read_csv('/Users/dB/Documents/repos/github/bint-capstone/data-sources/df_scaled_turnovers.csv')
ast_df = pd.read_csv('/Users/dB/Documents/repos/github/bint-capstone/data-sources/df_scaled_assists.csv')

In [6]:
pts_df.head()

,rebounds_total,rebounds_offensive,three_pointers_percentage,passes,assist_ratio,free_throws_attempted,field_goals_attempted,estimated_defensive_rating,defensive_rebound_percentage,uncontested_field_goals_percentage,...,rebound_percentage,blocks,true_shooting_percentage,estimated_pace,assist_to_turnover,pace,defensive_rating,offensive_rebound_percentage,contested_field_goal_percentage,assists
0,-0.575796,-1.135701,1.143999,2.038227,0.944887,0.036112,-1.151552,-1.269472,0.367155,0.468428,...,0.157283,-0.331987,0.482557,-1.446284,1.598707,-1.725084,-0.995316,-0.868839,-0.185888,0.240644
1,-2.550869,-1.135701,0.567548,-0.490530,-1.642507,-0.375301,-2.092907,1.815185,-3.774630,-0.031509,...,-2.803681,-1.529692,-0.330435,-2.787959,-1.063684,-2.591398,1.540368,-2.752196,-0.264480,-2.044974
2,-2.702798,-1.398928,1.296269,-1.038935,-0.644958,0.173250,-3.303220,1.815185,0.273275,1.037801,...,0.088423,-1.529692,1.183962,-2.425140,-1.251176,-2.398884,1.685998,-0.329226,0.285665,-1.283102
3,-1.031582,0.180433,1.143999,1.063285,0.851367,-0.375301,-0.613635,0.085391,-3.774630,0.149024,...,-2.803681,-1.529692,-0.489845,-1.725957,0.636247,-1.821341,0.058364,-0.001225,-0.736033,0.431112
4,-0.727725,-1.398928,1.774832,1.307020,1.693049,-0.101026,-2.496345,-0.545174,0.720588,0.426767,...,0.833359,-0.331987,2.363598,-0.924731,-0.688699,-0.569998,-0.806853,-0.022387,2.564838,1.192985


In [10]:
print(pts_df.columns.tolist())

['rebounds_total', 'rebounds_offensive', 'three_pointers_percentage', 'passes', 'assist_ratio', 'free_throws_attempted', 'field_goals_attempted', 'estimated_defensive_rating', 'defensive_rebound_percentage', 'uncontested_field_goals_percentage', 'free_throw_assists', 'plus_minus_points', 'estimated_offensive_rating', 'turnover_ratio', 'rebounds_defensive', 'contested_field_goals_made', 'steals', 'estimated_usage_percentage', 'free_throws_made', 'fouls_personal', 'effective_field_goal_percentage', 'contested_field_goals_attempted', 'offensive_rating', 'three_pointers_attempted', 'points', 'rebound_chances_defensive', 'p_i_e', 'estimated_team_turnover_percentage', 'opponent_points', 'rebound_percentage', 'blocks', 'true_shooting_percentage', 'estimated_pace', 'assist_to_turnover', 'pace', 'defensive_rating', 'offensive_rebound_percentage', 'contested_field_goal_percentage', 'assists']


In [7]:
to_df.head()

,rebounds_offensive,three_pointers_percentage,passes,assist_ratio,free_throws_attempted,uncontested_field_goals_attempted,field_goals_attempted,estimated_defensive_rating,defensive_rebound_percentage,touches,...,p_i_e,estimated_team_turnover_percentage,rebound_percentage,true_shooting_percentage,estimated_pace,pace,defensive_rating,offensive_rebound_percentage,possessions,turnovers
0,-1.135701,1.143999,2.038227,0.944887,0.036112,-1.277025,-1.151552,-1.269472,0.367155,1.568215,...,1.371537,-1.363854,0.157283,0.482557,-1.446284,-1.725084,-0.995316,-0.868839,-1.646399,-1.409342
1,-1.135701,0.567548,-0.490530,-1.642507,-0.375301,-1.148173,-2.092907,1.815185,-3.774630,-0.943649,...,-2.282225,0.255896,-2.803681,-0.330435,-2.787959,-2.591398,1.540368,-2.752196,-2.330851,-0.115465
2,-1.398928,1.296269,-1.038935,-0.644958,0.173250,-2.436696,-3.303220,1.815185,0.273275,-1.390202,...,-1.517746,2.344045,0.088423,1.183962,-2.425140,-2.398884,1.685998,-0.329226,-2.330851,1.695962
3,0.180433,1.143999,1.063285,0.851367,-0.375301,-1.019321,-0.613635,0.085391,-3.774630,0.703017,...,-0.247361,-0.504059,-2.803681,-0.489845,-1.725957,-1.821341,0.058364,-0.001225,-1.817512,-0.633016
4,-1.398928,1.774832,1.307020,1.693049,-0.101026,-1.792434,-2.496345,-0.545174,0.720588,0.842565,...,1.416506,2.661888,0.833359,2.363598,-0.924731,-0.569998,-0.806853,-0.022387,-0.619722,2.213513


In [11]:
print(to_df.columns.tolist())

['rebounds_offensive', 'three_pointers_percentage', 'passes', 'assist_ratio', 'free_throws_attempted', 'uncontested_field_goals_attempted', 'field_goals_attempted', 'estimated_defensive_rating', 'defensive_rebound_percentage', 'touches', 'free_throw_assists', 'estimated_offensive_rating', 'turnover_ratio', 'contested_field_goals_made', 'steals', 'estimated_usage_percentage', 'free_throws_made', 'fouls_personal', 'effective_field_goal_percentage', 'offensive_rating', 'three_pointers_attempted', 'points', 'three_pointers_made', 'rebound_chances_defensive', 'p_i_e', 'estimated_team_turnover_percentage', 'rebound_percentage', 'true_shooting_percentage', 'estimated_pace', 'pace', 'defensive_rating', 'offensive_rebound_percentage', 'possessions', 'turnovers']


In [8]:
ast_df.head()

,rebounds_total,rebounds_offensive,three_pointers_percentage,passes,assist_ratio,free_throws_attempted,field_goals_attempted,estimated_defensive_rating,defensive_rebound_percentage,uncontested_field_goals_percentage,...,rebound_percentage,blocks,true_shooting_percentage,estimated_pace,assist_to_turnover,pace,defensive_rating,offensive_rebound_percentage,contested_field_goal_percentage,assists
0,-0.575796,-1.135701,1.143999,2.038227,0.944887,0.036112,-1.151552,-1.269472,0.367155,0.468428,...,0.157283,-0.331987,0.482557,-1.446284,1.598707,-1.725084,-0.995316,-0.868839,-0.185888,0.240644
1,-2.550869,-1.135701,0.567548,-0.490530,-1.642507,-0.375301,-2.092907,1.815185,-3.774630,-0.031509,...,-2.803681,-1.529692,-0.330435,-2.787959,-1.063684,-2.591398,1.540368,-2.752196,-0.264480,-2.044974
2,-2.702798,-1.398928,1.296269,-1.038935,-0.644958,0.173250,-3.303220,1.815185,0.273275,1.037801,...,0.088423,-1.529692,1.183962,-2.425140,-1.251176,-2.398884,1.685998,-0.329226,0.285665,-1.283102
3,-1.031582,0.180433,1.143999,1.063285,0.851367,-0.375301,-0.613635,0.085391,-3.774630,0.149024,...,-2.803681,-1.529692,-0.489845,-1.725957,0.636247,-1.821341,0.058364,-0.001225,-0.736033,0.431112
4,-0.727725,-1.398928,1.774832,1.307020,1.693049,-0.101026,-2.496345,-0.545174,0.720588,0.426767,...,0.833359,-0.331987,2.363598,-0.924731,-0.688699,-0.569998,-0.806853,-0.022387,2.564838,1.192985


In [12]:
print(ast_df.columns.tolist())

['rebounds_total', 'rebounds_offensive', 'three_pointers_percentage', 'passes', 'assist_ratio', 'free_throws_attempted', 'field_goals_attempted', 'estimated_defensive_rating', 'defensive_rebound_percentage', 'uncontested_field_goals_percentage', 'free_throw_assists', 'plus_minus_points', 'estimated_offensive_rating', 'turnover_ratio', 'rebounds_defensive', 'contested_field_goals_made', 'steals', 'estimated_usage_percentage', 'free_throws_made', 'fouls_personal', 'effective_field_goal_percentage', 'contested_field_goals_attempted', 'offensive_rating', 'three_pointers_attempted', 'points', 'rebound_chances_defensive', 'p_i_e', 'estimated_team_turnover_percentage', 'opponent_points', 'rebound_percentage', 'blocks', 'true_shooting_percentage', 'estimated_pace', 'assist_to_turnover', 'pace', 'defensive_rating', 'offensive_rebound_percentage', 'contested_field_goal_percentage', 'assists']


In [ ]:
pts_x = pts_df.drop(['points'], axis=1)
pts_y = pts_df['points']

ast_x = ast_df.drop(['turnovers'], axis=1)
ast_y = ast_df['turnovers']

to_x = to_df.drop(['points'], axis=1)
to_y = to_df['points']

KeyError: "['points'] not found in axis"

# Removing Near-Zero Variance and Multicollinearity

The following functions will remove multicollinearity between independent variables and near-zero variance in predictors.

In [4]:
'''
This function drops highly correlated variables given a threshold.
It drops the variable with smaller variance.

It returns:
1. The variables that were dropped
2. The data without the dropped variables
'''

def drop_highly_correlated(data, threshold=.7):
    from dython.nominal import associations
    cor_matrix = associations(dataset=data, compute_only=True)['corr'].abs()
    #cor_matrix = data.corr().abs()
    sol = (cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(bool))).unstack().dropna()

    to_drop=[]
    for i in range(len(sol[sol > threshold].index.tolist())):
        var1 = sol[sol >= threshold].index.tolist()[i][0]
        var2 = sol[sol >= threshold].index.tolist()[i][1]

        if(data[var1].var() < data[var2].var()): 
            to_drop.append(var1)
        else: 
            to_drop.append(var2)

    dropped_vars = list(set(to_drop))
    return dropped_vars, data.drop(dropped_vars, axis=1) # (list of dropped vars, resultant df)

In [3]:
'''
remove_nzv_features removes features in the DataFrame with a variance below the parameter threshold. 
This is done by creating a mask using the sklearn VarianceThreshold function.

returns a tuple of the dataframe with the selected features (retained_df) and a dataframe of the dropped features (dropped_df)
'''
def remove_nzv_features(df:pd.DataFrame, threshold=.1): 
    from sklearn.feature_selection import VarianceThreshold

    vThreshold = VarianceThreshold(threshold)
    vThreshold.fit_transform(df)

    # Get boolean mask for selected columns
    mask = vThreshold.get_support()
    
    # Select columns to keep and drop
    selected_columns = df.columns[mask]
    dropped_columns = df.columns[~mask]
    
    # Create DataFrames for retained and removed features
    retained_df = df[selected_columns]
    dropped_df = df[dropped_columns]

    return retained_df, dropped_df

Finally, this produces a correlation plot of the results of removing the misleading features.

In [5]:
def my_corrplot(df, dep_var):
    import seaborn as sns
    from dython.nominal import associations

    # create a correlation matrix of all predictor variables using 'associations'
    corr_matrix = associations(dataset=df.drop(dep_var, axis=1), compute_only=True)['corr']

    # Mask upper triangle of the corr heatmap ???
    mask_upper = np.triu(np.ones_like(corr_matrix, dtype=bool))

    # Plot
    plt.figure(figsize=(44,44))
    sns.heatmap(corr_matrix, annot=True, mask=mask_upper, fmt='.2f', vmin=-1, vmax=1)
    plt.show()

Now we can apply these functions to the data.